In [ ]:

import pandas as pd
import numpy as np

#upload the file /Users/tommi/Desktop/Uni_documents/Data/commodity-2005.csv taking the "Dates" column as index
commodity = pd.read_csv('/Users/tommi/Desktop/Uni_documents/Data/commodity-2005.csv', index_col='Dates')